# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** I believe the problem given here is more of a Classification. Since we would be evaluating the student based on some features and after that we would use it to classify ass 0 or 1. 1 meaning needing intervention and 0 otherwise.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [9]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# student_data.info()
# student_data.describe()
# student_data.head(5)

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [10]:
# TODO: Calculate number of students
n_students = len(student_data.index) 

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1
#subtracted 1 for the column which was the predicted value/label.

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed']=='yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed']=='no'])

# TODO: Calculate graduation rate
grad_rate = (float)(n_passed*100)/(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [11]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

# Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [12]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [13]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train,X_test,y_train,y_test = train_test_split(X_all,y_all,test_size = 0.24,random_state = 1)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** The three supervised learning models appropriate for this problem are :-

**1. Random Forest : -**
Decision Trees are known for many uses like for drug analysis using tree based classifciation. Decision Trees are known to overfit and generally have problem dealing with noises. 
This is where Random Forest is used where random number of trees are generated with random number of features and then the final tree is produced by combining the previous trees which were weighted based on their accuracy. This usually boosts the performance wherever Decision Trees are used. Random forest can also do bad if our individual trees are compex. One more problem will be the time complexity , since we need to train individual trees and then average the result based on weights. However after the training phase is over it won't be requiring more time again. This model will work good because here we don't have a small data , and this is a classification problem with quite a number of features which will provide nice Information Gain. And as for overfitting Randomization with weighted trees will be dealing with it mostly.
Refrence : - http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html

**2. Support Vector Machines : -** 
Support Vector Machines. One of the most frequent uses of SVM is in text/handwriting recognition, since the underlying conept can be difficult. The strengths of such model is it can capture the underlying concept very well since we can change kernels from linear( Similar to Logistic Regression ) to RBF , specify the maxmimum degree of the polynomial function , which finally returns the best possible function. So capturing difficult concepts is possible. The data available has many features so there might be a possibility of a difficult concept. The weakness of this model is that it takes a LOT of time to train. If a lot of data is available this is one of the worst approaches in terms of time/space complexity. So it is usually avoided if a large amount of data is available. You can split the training and testing data to minimize the training data but there is a slight chance that it might then capture the wrong concept , and would actually be a bad model. Here only 300 samples are there so it's quite easy to use SVM and capture easy/hard concept.
Reference : - https://en.wikipedia.org/wiki/Support_vector_machine#Applications


**3. Weighted K-N-N : -**
KNN is a quite popular algorithm since , it usually is the case that a data point lying near to a specific with points with a particular label , will be of that label. Of course there might be noise in that data , but weighted K-N-N will be of help over there. Some practical uses are : Price of land/housing in a region can be predicted if it's near a rich locality etc; predicting if a region is prone to certain disasters , and many more. The strengths of this model is that it is robust to noise since the distance function that we need to define can be adjusted accordingly. Moreover the training time is very short. As for weaknesses the computational cost is high due to the fact that we need to calculate distance of a point from its k nearest neightbours. Moreover it's sometimes hard to identify the best value of K for the model. The space complexity is high too , since we need to store the weights, distance and all the data until we finish predicting. For the given problem I believe that this model would do suitably well , since it's often the case that if a person is in close proximity with a certain set of persons exhibiting common features then he is often labelled in the same way. Generally speaking if a person is in a group of failures he has a high chance of failing too. Reference : - https://saravananthirumuruganathan.wordpress.com/2010/05/17/a-detailed-introduction-to-k-nearest-neighbor-knn-algorithm/

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [14]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [17]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
# from sklearn import model_A
from sklearn.neighbors import KNeighborsClassifier
# from sklearn import model_B
from sklearn.ensemble import RandomForestClassifier
# from skearln import model_C

# TODO: Initialize the three models
clf_A = SVC(random_state = 40)
clf_B = KNeighborsClassifier()
clf_C = RandomForestClassifier(random_state= 40)

# TODO: Set up the training set sizes
X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)

X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train.head(300)
y_train_300 = y_train.head(300)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for clf in [clf_A, clf_B, clf_C]:
    for size in [100,200,300]:
        train_predict(clf,X_train[:size],y_train[:size],X_test,y_test)

Training a SVC using a training set size of 100. . .
Trained model in 0.0050 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8591.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8333.
Training a SVC using a training set size of 200. . .
Trained model in 0.0040 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8581.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8408.
Training a SVC using a training set size of 300. . .
Trained model in 0.0090 seconds
Made predictions in 0.0090 seconds.
F1 score for training set: 0.8584.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.8462.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0040 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.7883.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7727.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.00

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Using Support Vector Machines **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0020s          |        0.0010s         |      0.8591      |    0.8333       |
| 200               |        0.0050s          |        0.0030s         |      0.8581      |    0.8408       |
| 300               |        0.0090s          |        0.0060s         |      0.8584      |    0.8462       |

** Classifer 2 - K-N-N Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0010s          |         0.0010s        |      0.7883      |    0.7727       |
| 200               |        0.0010s          |         0.0040s        |      0.8345      |    0.7971       |
| 300               |        0.0020s          |         0.0090s        |      0.8558      |    0.7681       |

** Classifer 3 - Using Random Forest(Ensemble)**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0270s          |         0.0020s        |      1.0000      |    0.6829       |
| 200               |        0.0260s          |         0.0010s        |      0.9886      |    0.7520       |
| 300               |        0.0180s          |         0.0010s        |      0.9949      |    0.6977       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Although according to my expectations Random Forest should have done the best , but to my surprise SVM is the better model here.

Deciding the constraints of the problem as follows :-
- Available Data will be minimum 100 samples and maximum can be anything. So for the minimum data SVM has the best F1 score among the three chosen models. Random Forest will get better if some parameters like depth of tree etc are tuned.
- Limited resources & Cost : Of course the cost will be higher because we can see as amount of data increases the training time increases quite largely. This would require a higher load on machine performing the analysis. This would further increase the cost. The most time/space efficient model is K-N-N as the training time is quite_low and prediction time is also not that high.
- Performance : SVM is the best model in terms of performance which is quite evident from the F1 score. SVM is quite good at capturing concepts with a good number of features or those which are complex and it gets better as amount of data increases. I think Random Forest will perform better than now if amount of data is increased.

Judging by the above characteristics SVM is the best model if cost is not an issue, moreover at some point in time , the population of student is going to increase which will force people to invest in the cost area of the machine to further balance the increase in population of students, and improve the model by training on more points. So considering all the aspects above and the future , I think investing now in a SVM based model will be the best thing since , it provides quite accurate result in not a large prediction time. The training time is a one time process and we will be more concerned about the testing_time. There is a large probability that the F1 score will subsequently increase with the population of students. The F1 score is necessary here becaue if the University decides to invest some special program for failing students to help them , can help since , if a model has high F1 score would correctly predict the students. This would allow to not invest wrongly in the falsely predicte students .

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** SVM is model that has an accuracy of predicting 85% correctly , which is quite good. SVM is used in situtations where data might not be linearly seperable , which might be the case here. A student's success or failure rate can depend on many factors and features. So there might be many situations where there is a complex relationship with the required variable value and the student's characteristics. To capture such an underlying concept a model with SVM is used. 

This problem involves splitting two type of studens - successful ones and the non successful ones. The model just divides the two types of students either by lines etc. Sometimes the problem is not so easily solved by a line. In such cases our SVM can handle the problem efficiently. Our problem has 30 features which is quite a lot and the way they depend on each other and the success/failure rate is quite difficult to capture. SVM can then one by one first divide based on 1 feature then the second and further proceeding on the last 30 features. The common solution is our answer. One more important thing is that SVM has a way of measuring how similar our current student is similar to the students already categorized. It then uses this similarity as a measure and places students in one of the two categories depending one how similar is that particular student to the two types of students.

After training from the above mentioned method it then just checks the relative position of the new testing point and just checks on which side of plane it lies. Since I am quite sure there will be difference between a successful student and a failing student so I don't need to worry about the two types of students being close to each other in terms of characteristics. There might be some exceptions , of course , for such cases SVM while training makes sure to maximize the distance between the two categories as much as possible.Even for exceptions there is a divider too which it tries to maximize its distance from the other two categories. As it might be evident training the model required a lot of time because capturing each of the features and making solutions to find the best one between the two set of labels will be time inefficient. It might be a cost to pay with such a high accuracy. However this time is only a one time burden , because it doesn't find the solutions again and again , just tests the new point on the best solution it found during the training time. As the number of data points increases , the time it takes to train also increase and so does the accuracy.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [80]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV 
from sklearn.svm import SVC
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune

parameters = {'kernel': ('linear', 'rbf', 'poly','sigmoid'), 'C':[1,1.2,1.4,1.6,1.8,2,3,4,5,6,7,8,9,10] , 'degree':[1,2,3,4,5,6,7] }

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,parameters,f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0050 seconds.
Tuned model has a training F1 score of 0.8834.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.8456.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final model's F1 score for training has increased quite a bit but for testing has very slightly decreased. This might be the case that there might be noise in the training data which the SVM captured as I tuned the parameters to a high level and might be overfitting to the data/noise in the training data. This further increased it's accuracy but decreased the score of testing data by a little value. The untuned model had a low complexity and was not that much affected by noise. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.